<a href="https://colab.research.google.com/github/mn5hk/2020a_SSH_mapping_NATL60/blob/master/Simple_CAMELS_streamflow_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MHPI hydroDL Tutorial

This code contains deep learning code used to model hydrologic systems from soil moisture to streamflow or from projection to forecast. If this work is useful to you, please cite


*   Feng, Dapeng, Kuai Fang and Chaopeng Shen, Enhancing streamflow forecast and extracting insights using continental-scale long-short term memory networks with data integration, Water Resources Research, doi: 10.1029/2019WR026793
*   Fang, Kuai and Chaopeng Shen, Near-real-time forecast of satellite-based soil moisture using long short-term memory with an adaptive data integration kernel, Journal of Hydrometeorology, JHM-D-19-0169.1, doi:10.1175/JHM-D-19-0169.1 (2020)



[![PyPI](https://img.shields.io/badge/pypi-version%200.1-blue)](https://pypi.org/project/hydroDL/0.1.0/)  [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3993880.svg)](https://doi.org/10.5281/zenodo.3993880) [![CodeStyle](https://img.shields.io/badge/code%20style-Black-black)]()


Welcome to our hydroDL tutorial at The Pennsylvania State University! The following notebook is designed to provide a quick start to our project and get you ready to write your own neural networks.

### Obtain hydroDL from Github

In [ ]:
import os
os.chdir("/content/")
import sys
sys.path.append('../')
!rm -rf hydroDLpack # removes any old versions of hydroDL (in case you ran this tutorial previously)
!git clone https://github.com/mhpi/hydroDL.git hydroDLpack
os.chdir("/content/hydroDLpack")
!conda develop .

Cloning into 'hydroDLpack'...
remote: Enumerating objects: 1165, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1165 (delta 116), reused 143 (delta 85), pack-reused 960
Receiving objects: 100% (1165/1165), 60.71 MiB | 24.98 MiB/s, done.
Resolving deltas: 100% (499/499), done.
/bin/bash: line 1: conda: command not found


### Define Data Download Functions


In [ ]:
# This block defines functions used to obtain the data used for training and validation of the model. Real download action is in the next block.
# If you want to use the code with your own data, you can skip this block.
# Because CAMELS data is large and we cannot redistribute it, we provide two options for getting the data
# (1) set ReDownloadCAMEL=True in the next block and run that block to download it directly via colab. This can be slow and sometimes, you get cut off by NCAR servers
# (2) set ReDownloadCAMEL=False. get a script (download_CAMELS_script.py with path below) to run locally, define training/validation periods and process them into pickle files.
#      Upload the pickle files into the colab session to continue.
#2 is much faster and more reliable, but requires you have a local python installation with some dependencies.

from hydroDL.master import default
from hydroDL.data import camels
import platform
import os
import requests
import zipfile

def downloadCAMELS():
  if platform.system() == 'Windows':
    def download_file(url, destination):
        """Download a file from a specified URL to a given destination."""
        response = requests.get(url, stream=True)
        with open(destination, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

    def unzip_file(source, destination):
        """Unzip a file from a source to a destination."""
        with zipfile.ZipFile(source, 'r') as zip_ref:
            zip_ref.extractall(destination)

    # Base directory
    base_dir = os.getcwd()

    # Create necessary directories
    os.makedirs(base_dir + 'camels_attributes_v2.0/', exist_ok=True)
    os.makedirs(base_dir + 'camels_attributes_v2.0/camels_attributes_v2.0/', exist_ok=True)

    # Download and unzip the main dataset
    main_dataset_url = 'https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip'
    main_dataset_dest = base_dir + 'basin_timeseries_v1p2_metForcing_obsFlow.zip'
    download_file(main_dataset_url, main_dataset_dest)
    unzip_file(main_dataset_dest, base_dir + 'basin_timeseries_v1p2_metForcing_obsFlow/')

    # List of other files to download
    files_to_download = {
        'https://gdex.ucar.edu/dataset/camels/file/camels_attributes_v2.0.xlsx': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_attributes_v2.0.xlsx',
        'https://gdex.ucar.edu/dataset/camels/file/camels_clim.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_geol.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_hydro.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_name.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_name.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_topo.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_vege.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt'
    }

    # Download additional files
    for url, dest in files_to_download.items():
        full_dest = os.path.join(base_dir, dest)
        download_file(url, full_dest)

    print("Download and extraction complete.")

# Add additional download and unzip commands as needed

  else:
    !wget 'https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip' -O '/content/basin_timeseries_v1p2_metForcing_obsFlow.zip'
    !unzip '/content/basin_timeseries_v1p2_metForcing_obsFlow.zip' -d '/content/basin_timeseries_v1p2_metForcing_obsFlow'
    !mkdir '/content/camels_attributes_v2.0/'
    !mkdir '/content/camels_attributes_v2.0/camels_attributes_v2.0/'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_attributes_v2.0.xlsx' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_attributes_v2.0.xlsx'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_clim.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_geol.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_hydro.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_name.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_name.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_topo.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_vege.txt' -O '/content/camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt'


def extractCAMELS(Ttrain,attrLst,varF,camels,forType='daymet',flow_regime=1,subset_train="All",file_path=None):
  # flow_regime==1: high flow expert procedures
  train_loader = camels.DataframeCamels(subset=subset_train, tRange=Ttrain, forType=forType)
  x = train_loader.getDataTs(varLst=varF, doNorm=False, rmNan=False, flow_regime=flow_regime)
  y = train_loader.getDataObs(doNorm=False, rmNan=False, basinnorm=False, flow_regime=flow_regime)
  c = train_loader.getDataConst(varLst=attrLst, doNorm=False, rmNan=False, flow_regime=flow_regime)
  # define dataset
  if file_path is not None:
    with open(file_path, 'wb') as f:
      pickle.dump((x, y, c), f)
  return x, y, c

loading package hydroDL


### Download Data (READ! Select how data is obtained)

In [ ]:
# If you want to re-download CAMELS from scratch, set ReDownloadCAMEL=True and run this cell
# WARNING: Due to NCAR server behaviors, LONG run-time may be needed (and if obtaining errors when loading dataset later, may need to check to ensure everything downloaded correctly/run again)
# A faster and more reliable approach is to get this code and run it locally (brief instructions on how to run the script are included):
# https://github.com/mhpi/hydroDL/blob/release/example/download_CAMELS_script.py
# This script defines training/validation time periods and variables etc
# You will get training_file and validation_file after running the above script. Upload them under hydroDLpack in the Colab session
ReDownloadCAMEL=False
if ReDownloadCAMEL:
  downloadCAMELS() #KL: should this be commented out by default then, if we're going to load from pickle file by default?
  justLoadXYC = False #KL: make true if using pickle files, false if downloading data directly
else:
  # Manually upload training_file and validation_file under hydroDLpack before proceeding!
  justLoadXYC = True #KL: make true if using pickle files, false if downloading data directly
  #KL: If you are starting with pickle files (or files you downloaded manually or using a script) instead of directly downloading the CAMELS data from the source:
  #KL: Click on the folder icon on the left-hand side, then select the option for file upload, and choose both pickle files. This seems to run faster than the alternative below.

  #KL: Alternatively, run the following lines of code and when prompted ("browse" button will show below), select the files to upload
  #KL: (if pickle files, will be named "training_file" and "validation_file"). Took us ~30 minutes.
  #from google.colab import files
  #uploaded = files.upload()
camels.initcamels



<function hydroDL.data.camels.initcamels(flow_regime, forType, rootDB='/scratch/Camels')>

### Define Statistics Functions and a Scaler

In [ ]:
# This part defines a hydrology-specific Scaler class that works similar as sklearn.MinMaxScaler
# getStatDic, calcStat, calcStatgamma are supporting functions
# If you want to use it with your own data, you can create your own scaler that can be used to normalize your data
# Later on, we will use this scaler to transform the train and val data

def getStatDic(log_norm_cols, attrLst=None, attrdata=None, seriesLst=None, seriesdata=None):
  statDict = dict()
  # series data
  if seriesLst is not None:
    for k in range(len(seriesLst)):
      var = seriesLst[k]
      if var in log_norm_cols:
        statDict[var] = calcStatgamma(seriesdata[:, :, k])
      else:
        statDict[var] = calcStat(seriesdata[:, :, k])

  # const attribute
  if attrLst is not None:
    for k in range(len(attrLst)):
      var = attrLst[k]
      statDict[var] = calcStat(attrdata[:, k])
  return statDict

def calcStat(x):
  a = x.flatten()
  b = a[~np.isnan(a)]
  p10 = np.percentile(b, 10).astype(float)
  p90 = np.percentile(b, 90).astype(float)
  mean = np.mean(b).astype(float)
  std = np.std(b).astype(float)
  if std < 0.001:
    std = 1
  return [p10, p90, mean, std]

def calcStatgamma(x):  # for daily streamflow and precipitation
  a = x.flatten()
  b = a[~np.isnan(a)]  # kick out Nan
  b = np.log10(
    np.sqrt(b) + 0.1
  )  # do some tranformation to change gamma characteristics
  p10 = np.percentile(b, 10).astype(float)
  p90 = np.percentile(b, 90).astype(float)
  mean = np.mean(b).astype(float)
  std = np.std(b).astype(float)
  if std < 0.001:
    std = 1
  return [p10, p90, mean, std]

def transNormbyDic( x_in, var_lst, stat_dict, log_norm_cols, to_norm):
  if type(var_lst) is str:
    var_lst = [var_lst]
  x = x_in.copy()
  out = np.full(x.shape, np.nan)
  for k in range(len(var_lst)):
    var = var_lst[k]
    stat = stat_dict[var]
    if to_norm is True:
      if len(x.shape) == 3:
        if var in log_norm_cols:
          x[:, :, k] = np.log10(np.sqrt(x[:, :, k]) + 0.1)
        out[:, :, k] = (x[:, :, k] - stat[2]) / stat[3]
      elif len(x.shape) == 2:
        if var in log_norm_cols:
          x[:, k] = np.log10(np.sqrt(x[:, k]) + 0.1)
        out[:, k] = (x[:, k] - stat[2]) / stat[3]
    else:
      if len(x.shape) == 3:
        out[:, :, k] = x[:, :, k] * stat[3] + stat[2]
        if var in log_norm_cols:
          out[:, :, k] = (np.power(10, out[:, :, k]) - 0.1) ** 2
      elif len(x.shape) == 2:
        out[:, k] = x[:, k] * stat[3] + stat[2]
        if var in log_norm_cols:
          out[:, k] = (np.power(10, out[:, k]) - 0.1) ** 2
  return out



class HydroScaler:
  def __init__(self, attrLst, seriesLst, xNanFill,log_norm_cols):
    self.log_norm_cols = log_norm_cols
    self.attrLst = attrLst
    self.seriesLst = seriesLst
    self.stat_dict = None
    self.xNanFill = xNanFill

  def fit(self, attrdata, seriesdata):
    self.stat_dict = getStatDic(
      log_norm_cols=self.log_norm_cols,
      attrLst=self.attrLst,
      attrdata=attrdata,
      seriesLst=self.seriesLst,
      seriesdata=seriesdata,
    )

  def transform(self, data, var_list,):

    norm_data = transNormbyDic(
      data, var_list, self.stat_dict, log_norm_cols = self.log_norm_cols, to_norm=True)

    return norm_data

  def fit_transform(self, attrdata, seriesdata):
    self.fit(attrdata, seriesdata)
    attr_norm = self.transform(attrdata, self.attrLst)
    series_norm = self.transform(seriesdata, self.seriesLst)
    return attr_norm, series_norm


# The function it only used for streamflow unit convert and nondimensionalization
def basinNorm(x, basinarea, meanprep, toNorm):
  nd = len(x.shape)
  if nd == 3 and x.shape[2] == 1:
    x = x[:, :, 0]  # unsqueeze the original 3 dimension matrix
  temparea = np.tile(basinarea, (1, x.shape[1]))
  tempprep = np.tile(meanprep, (1, x.shape[1]))
  if toNorm is True:
    flow = (x * 0.0283168 * 3600 * 24) / (
            (temparea * (10 ** 6)) * (tempprep * 10 ** (-3))
    )  # (m^3/day)/(m^3/day)
  else:

    flow = (
            x
            * ((temparea * (10 ** 6)) * (tempprep * 10 ** (-3)))
            / (0.0283168 * 3600 * 24)
    )
  if nd == 3:
    flow = np.expand_dims(flow, axis=2)
  return flow

### Initialize hydroDL Library, Set Hyperparameters

In [ ]:
import torch
import time
import os
from tqdm import trange
import random
import hydroDL.core.logger as logger
from hydroDL.master.master import loadModel, wrapMaster, writeMasterFile
from hydroDL.model.rnn import CudnnLstmModel, CpuLstmModel
from hydroDL.model.crit import RmseLoss, NSELossBatch,NSESqrtLossBatch
from hydroDL.master import default
from hydroDL.data import camels
import numpy as np
import json
log = logger.get_logger("model.train_.train")

rootDatabase = "/content/"
device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device("cpu" )


# Fix random seed
seedid = 111111
random.seed(seedid)
torch.manual_seed(seedid)
np.random.seed(seedid)
torch.cuda.manual_seed(seedid)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set hyperparameters
EPOCH = 20 #KL: Reduced from 300 to 20 to save time/computational power when testing
BATCH_SIZE = 100
RHO = 365
HIDDENSIZE = 256
saveEPOCH = 20  # save model for every "saveEPOCH" epochs
trainBuff = 365
loadTrain = True
subset_train = 'All'  # give the list of basins to train on or else fix 'All' to use all

### Load Data

In [ ]:
"""
Dataset-specific code to read and scale the data
IF YOU CAN DIRECTLY PROVIDE DATA for train_x, train_y, train_c, val_x, val_y, val_c and scale the data
you can skip this section
The data structure is as follows:
train_x (forcing data, e.g. precipitation, temperature ...), shape: [pixels, time, features]
train_c (constant data, e.g. soil properties, land cover ...), shape: [pixels, features]
target/train_y (e.g. soil moisture, streamflow ...), shape: [pixels, time, 1]
val_x, val_c, val_y
Data type: numpy.float
The Scaler does
attr_norm, series_norm = scaler.fit_transform(train_c, series_data)
val_c = scaler.transform(val_c, attrLst)
val_x = scaler.transform(val_x, varF)
"""

import pickle

# These variables below define some options
# If you used download_CAMELS_script.py as described above. Some of these options are repeated in there
# However, they are put here because
forType = 'daymet'
flow_regime = 1
Ttrain = [19801001, 19951001] #training period
valid_date = [19951001, 20101001]  # Testing period
#define inputs
if forType == 'daymet':
  varF = ['dayl', 'prcp', 'srad', 'tmean', 'vp']
else:
  varF = ['dayl', 'prcp', 'srad', 'tmax', 'vp']

train_file = 'training_file'
validation_file = 'validation_file'
# Define attributes list
attrLst = [ 'p_mean','pet_mean', 'p_seasonality', 'frac_snow', 'aridity', 'high_prec_freq', 'high_prec_dur',
            'low_prec_freq', 'low_prec_dur', 'elev_mean', 'slope_mean', 'area_gages2', 'frac_forest', 'lai_max',
            'lai_diff', 'gvf_max', 'gvf_diff', 'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50',
            'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity',
            'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'geol_1st_class', 'glim_1st_class_frac',
            'geol_2nd_class', 'glim_2nd_class_frac', 'carbonate_rocks_frac', 'geol_porostiy', 'geol_permeability']

opt_data = default.optDataCamels # wrapping options around.
opt_data = default.update(opt_data, varT=varF, varC=attrLst, tRange=Ttrain, forType=forType, subset = subset_train)

if justLoadXYC:
    # Load X, Y, C from a file
    with open(train_file, 'rb') as f:
        train_x, train_y, train_c = pickle.load(f)  # Adjust this line based on your file format
    with open(validation_file, 'rb') as g:
        val_x, val_y, val_c = pickle.load(g)  # Adjust this line based on your file format
else:
  # a CAMELS dataset-specific object to manage basin normalization
  # If you are not using CAMELS, you can discard this one
  camels.initcamels(flow_regime=flow_regime, forType=forType, rootDB=rootDatabase)
  train_x, train_y, train_c = extractCAMELS(Ttrain,attrLst,varF,camels,forType='daymet',flow_regime=1,subset_train="All",file_path=train_file)
  val_x, val_y, val_c = extractCAMELS(valid_date,attrLst,varF,camels,forType='daymet',flow_regime=1,subset_train="All",file_path=validation_file)


# get a Scaler (similar to sklearn.MinMaxScaler).
# basinNorm function is only for converting the streamflow unit from ft^3/s to dimensionless.  The basin area and annual mean of precipiataion are required. y_temp = train_y/(basinarea*meanprep)
# If your streamflow is not in the unit of ft^3/s, this function needs to be adapted
basinarea  = val_c[:,np.where(np.array(attrLst)=='area_gages2')[0]]
meanprep  = val_c[:,np.where(np.array(attrLst)=='p_mean')[0]]
##Dimensionless streamflow
y_temp = basinNorm(train_y,  basinarea, meanprep, toNorm=True)

#train_x[np.isnan(train_x)] = 0.0 #KL: what is this? provide comment/context for when this should be uncommented
series_data = np.concatenate([train_x, y_temp], axis=2)
series_var_list = varF + ["runoff"]

# Initialize scaler
# This scaler will be used to scale the training data and later for the test as well.
# If flow_regime = 0, we will do a log transform and standard normalization on precipitation and streamflow; the other variables only use the standard normalization.
# If flow_regime = 1, we will only do standard normalization on all variables.
if flow_regime == 0:
    log_norm_cols = [ "prcp", "usgsFlow", "Precip", "runoff",  "Runoff", "Runofferror" ]
else:
    log_norm_cols = []

#train_c[np.isnan(train_c)] = 0.0 #KL: what is this? provide comment/context for when this should be uncommented
# Calculate the statistics for scaling the training data; Same statistics should be used for validation data for consistency. D
scaler = HydroScaler(attrLst=attrLst, seriesLst=series_var_list, xNanFill=0.0, log_norm_cols=log_norm_cols)
# Fit and transform training data


attr_norm, series_norm = scaler.fit_transform(train_c, series_data)
attr_norm[np.isnan(attr_norm)] = 0.0

train_x = series_norm[:, :, :-1]  # Forcing data
train_x[np.isnan(train_x)] = 0.0
train_y = np.expand_dims(series_norm[:, :, -1], 2)
train_c = attr_norm

# Fit and transform validation data

val_c = scaler.transform(val_c, attrLst)
val_x = scaler.transform(val_x, varF)
# no need to scale val_y before we transform the prediction back to compare with val_y

# fill some gaps in x. A fill for the normalized data is equal to filling with mean values
# We don't fill y because it will be handled by the loss function
val_c[np.isnan(val_c)] = 0.0
val_x[np.isnan(val_x)] = 0.0

#IF YOU CAN DIRECTLY PROVIDE DATA for train_x, train_y, train_c, val_x, val_y, val_c, and a Scaler and scale the data
# you can skip this section
#later code used scaler.stat_dict that matches with this scaler for testing. That could be changed for your data

In [ ]:
print(type(train_x))
print(train_x.shape, train_y.shape, train_c.shape)

<class 'numpy.ndarray'>
(671, 5478, 5) (671, 5478, 1) (671, 35)


### Intialize some options

In [ ]:
# define model and update configure
if torch.cuda.is_available():
    opt_model = default.optLstm
else:
    opt_model = default.update(default.optLstm, name="hydroDL.model.rnn.CpuLstmModel")
opt_model = default.update(default.optLstm, hiddenSize=HIDDENSIZE)

# this goes with your choice of flow_regime
if flow_regime==0:
    optLoss = default.optLossRMSE
elif flow_regime==1:
    optLoss = default.optLossNSEBatch

opt_train = default.update(default.optTrainCamels, miniBatch=[BATCH_SIZE, RHO], nEpoch=EPOCH, saveEpoch=1, seed=seedid, trainBuff=trainBuff)

# define output folder for model results
exp_name = f"CAMELSDemo"
exp_disp = "TestRun"
save_path = os.path.join(
    exp_name,
    exp_disp,
    "epochs{}_batch{}_rho{}_hiddensize{}_Tstart{}_Tend{}_trainBuff{}_flowregime{}".format(
        opt_train["nEpoch"],
        opt_train["miniBatch"][0],
        opt_train["miniBatch"][1],
        opt_model["hiddenSize"],
        opt_data["tRange"][0],
        opt_data["tRange"][1],
        opt_train['trainBuff'],
        flow_regime,
    ),
)
rootOut = os.path.join(
    os.sep, "content", "output", "rnnStreamflow"
)
output_path = os.path.join(rootOut, save_path, "All")

In [ ]:
# define training options

nx = train_x.shape[-1] + train_c.shape[-1]  # update nx, nx = nx + nc
ny = train_y.shape[-1]
#load model for training
if torch.cuda.is_available():
    model = CudnnLstmModel(nx=nx, ny=ny, hiddenSize=HIDDENSIZE).to(device)
else:
    model = CpuLstmModel(nx=nx, ny=ny, hiddenSize=HIDDENSIZE).to(device)

opt_model = default.update(opt_model, nx=nx, ny=ny)
if flow_regime==0:
  lossFun = RmseLoss()
elif flow_regime==1:
  lossFun = NSELossBatch(np.nanstd(train_y, axis=1),device=device)

# update and write the dictionary variable to out folder for logging and future testing
opt_all = wrapMaster(output_path, opt_data, opt_model, optLoss, opt_train)
writeMasterFile(opt_all)

write master file /content/output/rnnStreamflow/CAMELSDemo/TestRun/epochs20_batch100_rho365_hiddensize256_Tstart19801001_Tend19951001_trainBuff365_flowregime1/All/master.json


'/content/output/rnnStreamflow/CAMELSDemo/TestRun/epochs20_batch100_rho365_hiddensize256_Tstart19801001_Tend19951001_trainBuff365_flowregime1/All'

In [ ]:
'''
Dataloader functions for the training function. These functions are typically embedded inside hydroDL package,
but they are pulled out here so that newcomers can understand them without packaging details.
They help to build the minibatch and copy to cuda() device
Here we extract a minibatch from inputs["x", "c", "y"] and wrap the extracted data to data["x", "yTrain"]
'''

def select_subset(
    x, iGrid, iT, rho, c=None, tupleOut=False, LCopt=False, bufftime=0, **kwargs
):
    """
    Selects a subset based on the grid given
    :param x:
    :param iGrid:
    :param iT:
    :param rho:
    :param c:
    :param tupleOut:
    :param LCopt:
    :param bufftime:
    :return:
    """
    nx = x.shape[-1]
    nt = x.shape[1]
    if x.shape[0] == len(iGrid):  # hack
        iGrid = np.arange(0, len(iGrid))  # hack
    if nt <= rho:
        iT.fill(0)
    batchSize = iGrid.shape[0]
    if iT is not None:
        # batchSize = iGrid.shape[0]
        xTensor = torch.zeros([rho + bufftime, batchSize, nx], requires_grad=False)
        for k in range(batchSize):
            temp = x[
                iGrid[k] : iGrid[k] + 1, np.arange(iT[k] - bufftime, iT[k] + rho), :
            ]
            xTensor[:, k : k + 1, :] = torch.from_numpy(np.swapaxes(temp, 1, 0))
    else:
        if LCopt is True:
            # used for local calibration kernel: FDC, SMAP...
            if len(x.shape) == 2:
                # Used for local calibration kernel as FDC
                # x = Ngrid * Ntime
                xTensor = torch.from_numpy(x[iGrid, :]).float()
            elif len(x.shape) == 3:
                # used for LC-SMAP x=Ngrid*Ntime*Nvar
                xTensor = torch.from_numpy(np.swapaxes(x[iGrid, :, :], 1, 2)).float()
        else:
            # Used for rho equal to the whole length of time series
            xTensor = torch.from_numpy(np.swapaxes(x[iGrid, :, :], 1, 0)).float()
            rho = xTensor.shape[0]
    if c is not None:
        nc = c.shape[-1]
        temp = np.repeat(
            np.reshape(c[iGrid, :], [batchSize, 1, nc]), rho + bufftime, axis=1
        )
        cTensor = torch.from_numpy(np.swapaxes(temp, 1, 0)).float()
        if tupleOut:
            if torch.cuda.is_available():
                xTensor = xTensor.cuda()
                cTensor = cTensor.cuda()
            out = (xTensor, cTensor)
        else:
            out = torch.cat((xTensor, cTensor), 2)
    else:
        out = xTensor
    if torch.cuda.is_available() and type(out) is not tuple:
        out = out.cuda()
    return out


def random_index(ngrid, nt, dimSubset, bufftime=0):
    """
    Finds a random place to start inside of the grids
    :param: ngrid: the number of grids
    :param: nt: the number of tiles
    :param: dimSubset: the dimensions of the subset
    :param: bufftime: a buffer
    :returns: the index of the grid and the index of the tile
    """
    batchSize, rho = dimSubset
    iGrid = np.random.randint(0, ngrid, [batchSize])
    iT = np.random.randint(0 + bufftime, nt - rho, [batchSize])
    return iGrid, iT

def save_model(outFolder, model, epoch, modelName="model"):
    modelFile = os.path.join(outFolder, modelName + "_Ep" + str(epoch) + ".pt")
    torch.save(model, modelFile)


def load_model(outFolder, epoch, modelName="model"):
    modelFile = os.path.join(outFolder, modelName + "_Ep" + str(epoch) + ".pt")
    model = torch.load(modelFile)
    return model

def load_data(inputs, settings):
    """

    :param inputs:
    :param settings:
    :return:
    """
    data = {}
    iGrid, iT = random_index(
        settings["ngrid"], settings["nt"], [settings["batchSize"], settings["rho"]], bufftime=settings['bufftime']
    )
    data["x"] = select_subset(inputs["x"], iGrid, iT, settings["rho"], c=inputs["c"], bufftime=settings['bufftime'])
    data["yTrain"] = select_subset(inputs["y"], iGrid, iT, settings["rho"])
    return data, iGrid

### Define Training Function

In [ ]:
from hydroDL.model.settings import make_train_settings
import pandas as pd
from time import sleep
from tqdm import trange
import hydroDL.core.logger as logger
from hydroDL.model import crit

def trainModel(
    model,
    x,
    y,
    c,
    lossFun,
    nEpoch=500,
    load_data=load_data,
    miniBatch=[100, 30],
    saveEpoch=100,
    saveFolder=None,
    mode="seq2seq",
    bufftime=0,
):
    optim = torch.optim.Adadelta(model.parameters())
    if saveFolder is not None:
        runFile = os.path.join(saveFolder, "run.csv")
        rf = open(runFile, "w+")
    inputs, settings = make_train_settings(model, x, y, c, nEpoch, miniBatch, bufftime,)

    if torch.cuda.is_available():
        lossFun = lossFun.cuda()

    #_model_ = ModelWrapper(model)  # wraps up so the model can accept a dict
    # for illustration in this notebook, this wrapping is disabled.
    with trange(1, nEpoch + 1) as pbar:
        for iEpoch in pbar:
            pbar.set_description(f"Training {model.name}")
            lossEp = 0
            t0 = time.time()
            for iIter in range(0, settings["nIterEp"]):
                model.zero_grad()
                dataDict, iGrid = load_data(inputs, settings)
                x = dataDict["x"]
                yp = model(x)

                if type(lossFun) in [NSELossBatch, NSESqrtLossBatch]:
                    loss = lossFun(yp[bufftime:, :, :], dataDict["yTrain"], igrid=iGrid)
                else:
                    loss = lossFun(yp[bufftime:, :, :], dataDict["yTrain"])
                    # Additional handling code if needed

                loss.backward()
                optim.step()
                lossEp = lossEp + loss.item()
                # if iIter % 1 == 0:
                #     print('Iter {} of {}: Loss {:.3f}'.format(iIter, settings["nIterEp"], loss.item()))
            lossEp = lossEp / settings["nIterEp"]
            loss_str = "Epoch {} Loss {:.3f} time {:.2f}".format(
                iEpoch, lossEp, time.time() - t0
            )
            print(loss_str)
            log.debug(loss_str)
            pbar.set_postfix(loss=lossEp)
            sleep(0.1)
            if saveFolder is not None:
                rf.write(loss_str + "\n")
                if iEpoch % saveEpoch == 0:
                    # save model
                    modelFile = os.path.join(
                        saveFolder, "model_Ep" + str(iEpoch) + ".pt"
                    )
                    torch.save(model, modelFile)
    if saveFolder is not None:
        rf.close()

    # return model
    return model #KL: added to test

### Model Training

In [ ]:
# training the model
model = trainModel(
    model,
    train_x,
    train_y,
    train_c,
    lossFun,
    nEpoch=EPOCH,
    miniBatch=[BATCH_SIZE, RHO],
    saveEpoch=saveEPOCH,
    saveFolder=output_path,
    bufftime=trainBuff
)

Training CudnnLstmModel:   0%|          | 0/20 [00:00<?, ?it/s]/content/hydroDLpack/hydroDL/model/rnn/CudnnLstm.py:105: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
Training CudnnLstmModel:   5%|▌         | 1/20 [00:44<13:58, 44.11s/it, loss=0.534]

Epoch 1 Loss 0.534 time 44.01


Training CudnnLstmModel:  10%|█         | 2/20 [01:27<13:02, 43.46s/it, loss=0.39]

Epoch 2 Loss 0.390 time 42.90


Training CudnnLstmModel:  15%|█▌        | 3/20 [02:10<12:20, 43.54s/it, loss=0.338]

...

Epoch 18 Loss 0.203 time 44.30


Training CudnnLstmModel:  95%|█████████▌| 19/20 [14:04<00:44, 44.33s/it, loss=0.203]

Epoch 19 Loss 0.203 time 44.13


Training CudnnLstmModel: 100%|██████████| 20/20 [14:48<00:00, 44.44s/it, loss=0.2]

Epoch 20 Loss 0.200 time 44.43


In [ ]:
#need to load model. Modify path if needed, for example with number of epochs used
model = torch.load(f'/content/output/rnnStreamflow/CAMELSDemo/TestRun/epochs20_batch100_rho365_hiddensize256_Tstart19801001_Tend19951001_trainBuff365_flowregime1/All/model_Ep{EPOCH}.pt')

### Intialize Testing

In [ ]:
from hydroDL.master.master import readMasterFile

# validate the result
# load validation datasets
# load your data. same as training data
valid_epoch = EPOCH  # choose the model to test after trained "valid_epoch" epoches
valid_batch = 50  # do batch forward to save GPU memory

# adding some buffer data to validation data
testTrainBuff = True
loadtrainBuff = 0
TestBuff = train_x.shape[1] - loadtrainBuff    # Testing period
if testTrainBuff is True:
  xTestBuff = train_x[:, -TestBuff:, :]
  val_x = np.concatenate([xTestBuff, val_x], axis=1)
model.zero_grad()

### Define Testing Function

In [ ]:
from hydroDL.model.settings import make_test_settings

def specify_inputs(inputs, settings, i):
    """
    Pulls x and c data based on i and the settings
    :return:
    """
    _inputs_ = inputs
    _inputs_["xTemp"] = inputs["x"][settings["iS"][i] : settings["iE"][i], :, :]
    if _inputs_["c"] is not None:
        _inputs_["cTemp"] = np.repeat(
            np.reshape(
                inputs["c"][settings["iS"][i] : settings["iE"][i], :],
                [settings["iE"][i] - settings["iS"][i], 1, settings["nc"]],
            ),
            settings["nt"],
            axis=1,
        )
        _inputs_["xTest"] = torch.from_numpy(
            np.swapaxes(np.concatenate([_inputs_["xTemp"], _inputs_["cTemp"]], 2), 1, 0)
        ).float()
    else:
        _inputs_["xTest"] = torch.from_numpy(
            np.swapaxes(_inputs_["xTemp"], 1, 0)
        ).float()
    if torch.cuda.is_available():
        _inputs_["xTest"] = _inputs_["xTest"].cuda()
    return _inputs_

from hydroDL.model.settings import make_test_settings
def testModel(
    model,
    x,
    c,
    *,
    batchSize=None,
    filePathLst=None,
    doMC=False,
    outModel=None,
    savePath=None,
):

    fLst = list()
    for filePath in filePathLst:
        if os.path.exists(filePath):
            os.remove(filePath)
        f = open(filePath, "a")
        fLst.append(f)
    inputs, settings = make_test_settings(model, x, c, batchSize, doMC, outModel)
    model.train(mode=False)
    for i in range(len(settings["iS"])):
        log.debug(f"batch {i}")
        data = specify_inputs(inputs, settings, i)
        print(f'iter {i}, working on test data dimension: ',data["xTest"].shape)
        yP = model(data["xTest"])

        # CP-- marks the beginning of problematic merge
        yOut = yP.detach().cpu().numpy().swapaxes(0, 1)

        # save output
        for k in range(settings["ny"]):
            f = fLst[k]
            pd.DataFrame(yOut[:, :, k]).to_csv(f, header=False, index=False)
        model.zero_grad()
        torch.cuda.empty_cache()

    for f in fLst:
        f.close()


### Model Testing and Performance

In [ ]:
from hydroDL import master, utils
from hydroDL.post import stat
print(valid_batch)
# load and forward the model for validation
test_model = loadModel(output_path, epoch=valid_epoch)
filePathLst = master.master.namePred(
    output_path, valid_date, "All", epoch=valid_epoch
)  # prepare the name of csv files to save testing results
testModel(test_model, val_x, c=val_c, batchSize=valid_batch, filePathLst=filePathLst)

pred = pd.read_csv(filePathLst[0], dtype=np.float, header=None).values[:, :, None]
# transform back to the original observation


50
read master file /content/output/rnnStreamflow/CAMELSDemo/TestRun/epochs20_batch100_rho365_hiddensize256_Tstart19801001_Tend19951001_trainBuff365_flowregime1/All/master.json
iter 0, working on test data dimension:  torch.Size([10957, 50, 40])


/content/hydroDLpack/hydroDL/model/rnn/CudnnLstm.py:105: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(


iter 1, working on test data dimension:  torch.Size([10957, 50, 40])
iter 2, working on test data dimension:  torch.Size([10957, 50, 40])
iter 3, working on test data dimension:  torch.Size([10957, 50, 40])
iter 4, working on test data dimension:  torch.Size([10957, 50, 40])
iter 5, working on test data dimension:  torch.Size([10957, 50, 40])
iter 6, working on test data dimension:  torch.Size([10957, 50, 40])
iter 7, working on test data dimension:  torch.Size([10957, 50, 40])
iter 8, working on test data dimension:  torch.Size([10957, 50, 40])
iter 9, working on test data dimension:  torch.Size([10957, 50, 40])
iter 10, working on test data dimension:  torch.Size([10957, 50, 40])
iter 11, working on test data dimension:  torch.Size([10957, 50, 40])
iter 12, working on test data dimension:  torch.Size([10957, 50, 40])
iter 13, working on test data dimension:  torch.Size([10957, 21, 40])


<ipython-input-15-5bcc6b03119f>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred = pd.read_csv(filePathLst[0], dtype=np.float, header=None).values[:, :, None]


In [ ]:

temppred = transNormbyDic(
      pred, "runoff", scaler.stat_dict, log_norm_cols = log_norm_cols, to_norm=False)

pred = basinNorm(temppred,  basinarea, meanprep, toNorm=False)

pred = pred[:,TestBuff:,:]* 0.0283168

obs = val_y* 0.0283168
# calculate statistic metrics
statDictLst = stat.statError(pred.squeeze(), obs.squeeze())
print(np.nanmedian(statDictLst["NSE"]))

0.722787794841875
